## 基于MindNLP的Bert模型应用开发
我们使用 Hugging Face 提供的预训练模型 `'huggingface-course/bert-finetuned-ner'` 来进行命名实体识别（NER）任务。此模型已经在特定的 NER 数据集上进行微调，能够自动识别文本中的实体类型，包括人物（PER）、地理位置（GEO）、组织（ORG）等。模型所使用的数据集是 Kaggle 的公开 NER 数据集，其中包含丰富的标签信息，帮助模型准确地理解和标记文本中的不同实体。

在模型推理过程中，给定的输入句子会被分词并标注，每个单词都带有对应的实体标签。例如，给定句子 `"Huawei is a company based in Shen Zhen, but it also has employees like Eric Zhang working in Singapore"`，模型输出结果如下：

- **原始句子分词**：`['Huawei', 'is', 'a', 'company', 'based', 'in', 'Shen', 'Zhen,', 'but', 'it', 'also', 'has', 'employees', 'like', 'Eric', 'Zhang', 'working', 'in', 'Singapore']`
- **预测标签**：`['B-org', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'I-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'I-per', 'O', 'O', 'B-geo']`

在这个示例中，模型成功地将 `"Huawei"` 标记为组织（`B-org`），将 `"Shen Zhen"` 和 `"Singapore"` 标记为地理位置（`B-geo` 和 `I-geo`），并将 `"Eric Zhang"` 标记为人物（`B-per` 和 `I-per`）。其他非实体词则被标记为 `O`，表示它们不是实体的一部分。


### **环境配置：**

1. MindSpore 2.3.1
2. Mindnlp 0.4.0
3. Python 3.9.0
4. kagglehub
5. pandas
6. numpy

**使用华为云 ModelArts 作为AI平台**

也可以在华为云 ModelArts上 直接跑本项目Notebook: https://pangu.huaweicloud.com/gallery/asset-detail.html?id=e12870f6-f88f-4011-bc8e-f99ce0a7e458

In [ ]:
!pip show mindspore
!pip show mindnlp

Name: mindspore
Version: 2.3.1
Summary: MindSpore is a new open source deep learning training/inference framework that could be used for mobile, edge and cloud scenarios.
Home-page: https://www.mindspore.cn
Author: The MindSpore Authors
Author-email: contact@mindspore.cn
License: Apache 2.0
Location: /home/ma-user/anaconda3/envs/mindnlp/lib/python3.9/site-packages
Requires: asttokens, astunparse, numpy, packaging, pillow, protobuf, psutil, scipy
Required-by: mindnlp
Name: mindnlp
Version: 0.4.0
Summary: An open source natural language processing research tool box. Git version: [sha1]:5b4dad33, [branch]: (HEAD -> master, ms/master)
Home-page: https://github.com/mindlab-ai/mindnlp/tree/master/
Author: MindSpore Team
Author-email: 
License: Apache 2.0
Location: /home/ma-user/anaconda3/envs/mindnlp/lib/python3.9/site-packages
Requires: addict, datasets, evaluate, jieba, mindspore, ml-dtypes, pillow, pyctcdecode, pytest, regex, requests, safetensors, sentencepiece, tokenizers, tqdm
Required

### 其他所需库安装

In [3]:
%pip install kagglehub
%pip install pandas

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Note: you may need to restart the kernel to use updated packages.


### 2 Bert 模型
原文地址：https://arxiv.org/abs/1810.04805

原TensorFlow代码：https://github.com/google-research/bert  
原Pytorch代码：https://github.com/codertimo/BERT-pytorch 

#### 2.1 Bert 模型

BERT（Bidirectional Encoder Representations from Transformers）是一种由谷歌提出的预训练语言模型，旨在通过双向 Transformer 编码器来捕获语言的上下文信息。与传统的单向语言模型不同，BERT 通过双向（从左到右和从右到左）同时处理文本，从而理解每个词在整个句子中的真正含义。这种方法能够更好地理解句子中的词义和结构，提高对自然语言的理解能力。BERT 模型的预训练采用了两个主要任务：**掩码语言模型（Masked Language Model，MLM）** 和 **下一个句子预测（Next Sentence Prediction，NSP）**。在 MLM 任务中，BERT 会随机掩盖输入中的部分词，并通过模型预测这些被掩盖的词，从而学习词汇和上下文之间的关系。而 NSP 任务则训练模型判断两个句子是否紧密关联，帮助模型理解句子间的逻辑关系。



#### 2.2 导入模型到mindnlp
可以从中 `dir(mindnlp.transformers)` 查看和导入到关于Bert的包

In [4]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"       #huggingface镜像      

In [5]:
import mindnlp
# dir(mindnlp.transformers)   

/home/ma-user/anaconda3/envs/mindnlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.626 seconds.
Prefix dict has been built successfully.


In [6]:
import numpy as np
from mindspore import Tensor
from mindnlp.transformers import BertTokenizerFast,BertForTokenClassification

# 初始化分词器和模型
tokenizer = BertTokenizerFast.from_pretrained('huggingface-course/bert-finetuned-ner')

/home/ma-user/anaconda3/envs/mindnlp/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


#### 2.4 加载数据
使用的是Kaggle 的 [`ner-dataset`](https://www.kaggle.com/datasets/namanj27/ner-dataset)，这是一个用于命名实体识别（NER）任务的公开数据集，主要用于训练和评估模型在识别文本中实体的能力。数据集包含大量已标注的文本，每个单词或标记（token）都带有相应的标签，表明它在文本中的角色，如一个示例行标签的含义如下：

### **BIO 标注格式**
- **B-** 表示实体的开始（Begin）。
- **I-** 表示实体的内部（Inside）。
- **O** 表示非实体（Outside）。

### **标签含义及示例**

| 标签       | 含义                                    | 示例句子                         | 解释                                             |
|------------|-----------------------------------------|----------------------------------|--------------------------------------------------|
| **O**      | Outside，非实体                         | "The sky is blue."               | "sky" 和 "blue" 都是 `O`，表示非实体词。          |
| **B-per**  | Begin-person，人物名称的开始            | "Albert Einstein was a physicist." | "Albert" 被标记为 `B-per`，表示人物的开始。        |
| **I-per**  | Inside-person，人物名称的内部词         | "Albert Einstein"                | "Einstein" 被标记为 `I-per`，表示人物的内部部分。   |
| **B-org**  | Begin-organization，组织名称的开始      | "Google is a tech company."      | "Google" 被标记为 `B-org`，表示组织的开始。       |
| **I-org**  | Inside-organization，组织名称的内部词   | "University of Oxford"           | "of" 和 "Oxford" 被标记为 `I-org`，表示组织名称的内部部分。 |
| **B-geo**  | Begin-geographical location，地理位置的开始 | "Paris is beautiful."            | "Paris" 被标记为 `B-geo`，表示一个地理位置的开始。 |
| **I-geo**  | Inside-geographical location，地理位置的内部词 | "San Francisco Bay"              | "Francisco" 和 "Bay" 被标记为 `I-geo`，表示地理位置内部部分。 |
| **B-misc** | Begin-miscellaneous，其他实体的开始      | "The Olympics are held every four years." | "Olympics" 被标记为 `B-misc`，表示其他实体的开始。 |
| **I-misc** | Inside-miscellaneous，其他实体的内部词   | "The 2020 Summer Olympics"       | "Summer" 和 "Olympics" 被标记为 `I-misc`，表示其他实体的内部部分。 |

### **具体示例解析**

给定以下句子：

> "Albert Einstein was born in Ulm, Germany and worked at the University of Zurich."

模型会给出的标签为：

| 单词       | 标签     | 解释                                  |
|------------|----------|---------------------------------------|
| Albert     | B-per    | `B-per` 表示人物名称的开始。           |
| Einstein   | I-per    | `I-per` 表示人物名称的内部词。         |
| was        | O        | `O` 表示非实体。                      |
| born       | O        | `O` 表示非实体。                      |
| in         | O        | `O` 表示非实体。                      |
| Ulm        | B-geo    | `B-geo` 表示地理位置的开始。          |
| ,          | O        | `O` 表示非实体标点符号。              |
| Germany    | B-geo    | `B-geo` 表示地理位置的开始。          |
| and        | O        | `O` 表示非实体。                      |
| worked     | O        | `O` 表示非实体。                      |
| at         | O        | `O` 表示非实体。                      |
| the        | O        | `O` 表示非实体。                      |
| University | B-org    | `B-org` 表示组织名称的开始。           |
| of         | I-org    | `I-org` 表示组织名称的内部词。         |
| Zurich.    | I-org    | `I-org` 表示组织名称的内部词。         |


In [7]:
import kagglehub

# 指定下载路径为当前目录
path = kagglehub.dataset_download("namanj27/ner-dataset")
print("Path to dataset files:", path)

100%|██████████| 3.17M/3.17M [00:02<00:00, 1.65MB/s]

Extracting files...


Path to dataset files: /home/ma-user/.cache/kagglehub/datasets/namanj27/ner-dataset/versions/2


In [8]:
import pandas as pd

data = pd.read_csv(path+"/ner_datasetreference.csv", encoding='unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


为模型准备数据

In [9]:
print("Number of tags: {}".format(len(data.Tag.unique())))
frequencies = data.Tag.value_counts()
frequencies

Number of tags: 17


Tag
O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: count, dtype: int64

In [10]:
entities_to_remove = ["B-art", "I-art", "B-eve", "I-eve", "B-nat", "I-nat"]
data = data[~data.Tag.isin(entities_to_remove)]
data = data.fillna(method='ffill')
# 创建一个名为 "sentence" 的新列，将单词按句子进行分组
# 创建一个名为 "word_labels" 的新列，将标签按句子进行分组
data['sentence'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
data['word_labels'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
data.head()

/tmp/ipykernel_2269/4071460696.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


,Sentence #,Word,POS,Tag,sentence,word_labels
0,Sentence: 1,Thousands,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
1,Sentence: 1,of,IN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
2,Sentence: 1,demonstrators,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
3,Sentence: 1,have,VBP,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
4,Sentence: 1,marched,VBN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."


创建两个字典：一个将每个标签映射到索引，另一个将索引映射到对应的标签。

In [11]:
label2id = {k: v for v, k in enumerate(data.Tag.unique())}
id2label = {v: k for v, k in enumerate(data.Tag.unique())}
label2id

{'O': 0,
 'B-geo': 1,
 'B-gpe': 2,
 'B-per': 3,
 'I-geo': 4,
 'B-org': 5,
 'I-org': 6,
 'B-tim': 7,
 'I-per': 8,
 'I-gpe': 9,
 'I-tim': 10}

按引号查看例子和标准Tag

In [12]:
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

,sentence,word_labels
0,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
1,Families of soldiers killed in the conflict jo...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-per,O,O,..."
2,They marched from the Houses of Parliament to ...,"O,O,O,O,O,O,O,O,O,O,O,B-geo,I-geo,O"
3,"Police put the number of marchers at 10,000 wh...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
4,The protest comes on the eve of the annual con...,"O,O,O,O,O,O,O,O,O,O,O,B-geo,O,O,B-org,I-org,O,..."


In [13]:
data.iloc[1].sentence

'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "'

##### 2.5 模型推理


定义模型必要的变量和初始化模型

In [14]:
label_list = ["O", "B-per", "I-per", "B-org", "I-org", "B-geo", "I-geo", "B-misc", "I-misc"]
labels_to_ids = {label: idx for idx, label in enumerate(label_list)}
num_labels = len(labels_to_ids)  
ids_to_labels = {v: k for k, v in labels_to_ids.items()}
model = BertForTokenClassification.from_pretrained('huggingface-course/bert-finetuned-ner', num_labels=num_labels)

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


In [15]:
def ner_prediction(sentence, model, tokenizer, ids_to_labels, max_length=128):
    """
    使用给定的 BERT 模型和分词器对输入句子进行命名实体识别。

    参数:
        sentence (str): 要进行命名实体识别的句子。
        model: 已加载的 BERT 模型，用于命名实体识别。
        tokenizer: 用于分词的 BERT 分词器。
        ids_to_labels (dict): 标签 ID 到标签名称的映射字典。
        max_length (int): 句子的最大长度，默认为 128。

    返回:
        list: 原始句子分词列表。
        list: 每个词对应的标签预测结果。
    """
    # 对句子进行分词
    inputs = tokenizer(
        sentence.split(),
        is_split_into_words=True,
        return_offsets_mapping=True,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors=None
    )

    # 将输入数据转换为 Tensors
    input_ids = np.array([inputs["input_ids"]], dtype=np.int32)
    attention_mask = np.array([inputs["attention_mask"]], dtype=np.int32)
    ids = Tensor(input_ids)
    mask = Tensor(attention_mask)

    # 模型前向传播
    outputs = model(input_ids=ids, attention_mask=mask)
    logits = outputs[0]

    # 获取预测结果
    active_logits = logits.reshape(-1, len(ids_to_labels))
    flattened_predictions = active_logits.argmax(axis=1)

    # 将预测的标签 ID 映射回标签名称
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
    token_predictions = [ids_to_labels[int(id)] for id in flattened_predictions.asnumpy()]

    # 提取首个子词的预测结果
    wp_preds = list(zip(tokens, token_predictions))
    offsets = inputs["offset_mapping"]

    prediction = []
    for token_pred, mapping in zip(wp_preds, offsets):
        if mapping[0] == 0 and mapping[1] != 0:
            prediction.append(token_pred[1])

    # 输出结果
    return sentence.split(), prediction

使用推理示例1-kaggle数据

In [16]:
# 输入句子
sentence = data.iloc[41].sentence

# 调用预测函数
words, predictions = ner_prediction(sentence, model, tokenizer, ids_to_labels)

# 打印输出
print("原始句子分词：", words)
print("预测标签：", predictions)

原始句子分词： ['Bedfordshire', 'police', 'said', 'Tuesday', 'that', 'Omar', 'Khayam', 'was', 'arrested', 'in', 'Bedford', 'for', 'breaching', 'the', 'conditions', 'of', 'his', 'parole', '.']
预测标签： ['B-geo', 'O', 'O', 'O', 'O', 'B-per', 'I-per', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


使用推理示例2-tag介绍例子和其他自定义例子

In [17]:
# 输入句子
sentence = "Albert Einstein was born in Ulm, Germany and worked at the University of Zurich."

# 调用预测函数
words, predictions = ner_prediction(sentence, model, tokenizer, ids_to_labels)

# 打印输出
print("原始句子分词：", words)
print("预测标签：", predictions)

原始句子分词： ['Albert', 'Einstein', 'was', 'born', 'in', 'Ulm,', 'Germany', 'and', 'worked', 'at', 'the', 'University', 'of', 'Zurich.']
预测标签： ['B-per', 'I-per', 'O', 'O', 'O', 'B-geo', 'B-geo', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'I-org']


In [18]:
# 输入句子
sentence = "Huawei is a company based in Shen Zhen, but it also has employees like Eric Zhang working in Singapore"

# 调用预测函数
words, predictions = ner_prediction(sentence, model, tokenizer, ids_to_labels)

# 打印输出
print("原始句子分词：", words)
print("预测标签：", predictions)

原始句子分词： ['Huawei', 'is', 'a', 'company', 'based', 'in', 'Shen', 'Zhen,', 'but', 'it', 'also', 'has', 'employees', 'like', 'Eric', 'Zhang', 'working', 'in', 'Singapore']
预测标签： ['B-org', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'I-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'I-per', 'O', 'O', 'B-geo']


In [19]:
# 输入句子
sentence = "@HuggingFace is a company based in New York, but it also has employees working in Paris"

# 调用预测函数
words, predictions = ner_prediction(sentence, model, tokenizer, ids_to_labels)

# 打印输出
print("原始句子分词：", words)
print("预测标签：", predictions)

原始句子分词： ['@HuggingFace', 'is', 'a', 'company', 'based', 'in', 'New', 'York,', 'but', 'it', 'also', 'has', 'employees', 'working', 'in', 'Paris']
预测标签： ['B-org', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'I-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo']
